In [1]:
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Tuple, List, Optional
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
import os
from langchain_community.graphs import Neo4jGraph
from langchain.document_loaders import WikipediaLoader
from langchain.text_splitter import TokenTextSplitter
from langchain_openai import ChatOpenAI
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_core.runnables import ConfigurableField, RunnableParallel, RunnablePassthrough
# LangChain supports many other chat models. Here, we're using Ollama
from langchain_community.chat_models import ChatOllama
from langchain_core.documents import Document
from dotenv import load_dotenv

In [2]:
load_dotenv()

graph = Neo4jGraph()

In [3]:
import pandas as pd
recipe_df = pd.read_csv("hf://datasets/Hieu-Pham/kaggle_food_recipes/Food Ingredients and Recipe Dataset with Image Name Mapping.csv")

In [4]:
recipe_df = recipe_df[['Title', 'Instructions', 'Cleaned_Ingredients']]
recipe_df.head()

,Title,Instructions,Cleaned_Ingredients
0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"Pat chicken dry with paper towels, season all ...","['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher..."
1,Crispy Salt and Pepper Potatoes,Preheat oven to 400°F and line a rimmed baking...,"['2 large egg whites', '1 pound new potatoes (..."
2,Thanksgiving Mac and Cheese,Place a rack in middle of oven; preheat to 400...,"['1 cup evaporated milk', '1 cup whole milk', ..."
3,Italian Sausage and Bread Stuffing,Preheat oven to 350°F with rack in middle. Gen...,"['1 (¾- to 1-pound) round Italian loaf, cut in..."
4,Newton's Law,Stir together brown sugar and hot water in a c...,"['1 teaspoon dark brown sugar', '1 teaspoon ho..."


#### text2graph

In [5]:
test_recipe = recipe_df.head()
test_recipe = test_recipe.to_dict(orient='records')

In [6]:
docs = []
for i in test_recipe:
    docs.append(str(i))

In [7]:
docs

['{\'Title\': \'Miso-Butter Roast Chicken With Acorn Squash Panzanella\', \'Instructions\': \'Pat chicken dry with paper towels, season all over with 2 tsp. salt, and tie legs together with kitchen twine. Let sit at room temperature 1 hour.\\nMeanwhile, halve squash and scoop out seeds. Run a vegetable peeler along ridges of squash halves to remove skin. Cut each half into ½"-thick wedges; arrange on a rimmed baking sheet.\\nCombine sage, rosemary, and 6 Tbsp. melted butter in a large bowl; pour half of mixture over squash on baking sheet. Sprinkle squash with allspice, red pepper flakes, and ½ tsp. salt and season with black pepper; toss to coat.\\nAdd bread, apples, oil, and ¼ tsp. salt to remaining herb butter in bowl; season with black pepper and toss to combine. Set aside.\\nPlace onion and vinegar in a small bowl; season with salt and toss to coat. Let sit, tossing occasionally, until ready to serve.\\nPlace a rack in middle and lower third of oven; preheat to 425°F. Mix miso and

In [8]:
documents = recipe_docs
# supports many more optional parameters. Hover on your `ChatOllama(...)`
# class to view the latest available supported parameters
llm = ChatOllama(model="llama3")
llm_transformer = LLMGraphTransformer(llm=llm)

graph_documents = llm_transformer.convert_to_graph_documents(documents)
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

NameError: name 'recipe_docs' is not defined

In [ ]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

[Relationship(source=Node(id='Miso-Butter Roast Chicken', type='Recipe'), target=Node(id='butter', type='Ingredient'), type='MADE_WITH'),
 Relationship(source=Node(id='Miso-Butter Roast Chicken', type='Recipe'), target=Node(id='miso', type='Ingredient'), type='MADE_WITH'),
 Relationship(source=Node(id='Miso-Butter Roast Chicken', type='Recipe'), target=Node(id='skillet', type='Cookware'), type='COOKED_IN'),
 Relationship(source=Node(id='Acorn Squash', type='Ingredient'), target=Node(id='bread mixture', type='Ingredient'), type='ROASTED_WITH'),
 Relationship(source=Node(id='Chicken Broth', type='Ingredient'), target=Node(id='Miso-Butter Roast Chicken', type='Recipe'), type='USED_IN'),
 Relationship(source=Node(id='White Miso', type='Ingredient'), target=Node(id='Miso-Butter Roast Chicken', type='Recipe'), type='USED_IN'),
 Relationship(source=Node(id='Butter', type='Ingredient'), target=Node(id='Miso-Butter Roast Chicken', type='Recipe'), type='USED_IN'),
 Relationship(source=Node(id='A

#### df2graph

In [9]:
test_recipe = recipe_df.head()

In [28]:
import re
import pandas as pd
import networkx as nx
from py2neo import Graph, Node, Relationship
import ast

# 初始化圖譜
G = nx.Graph()

# 提取成分名稱和份量的輔助函數
def parse_ingredients(ingredient_string):
    ingredients = ast.literal_eval(ingredient_string)
    parsed_ingredients = []
    for ingredient in ingredients:
        # 使用正則表達式來拆解成分名稱和份量
        match = re.match(r"([\d\/\s\.\½]+)?\s*(oz\.|teaspoons?|tablespoons?|cups?|lb\.|g|kg|ml|l)?\s*(.*)", ingredient.strip())
        if match:
            amount, unit, name = match.groups()
            amount = amount.strip() if amount else '1'
            unit = unit.strip() if unit else ''
            name = name.strip()
            # 處理數量的簡單轉換
            if '½' in amount:
                amount = amount.replace('½', '0.5')
            try:
                amount = float(eval(amount))
            except:
                amount = 1
            parsed_ingredients.append((name, f"{amount} {unit}".strip()))
        else:
            parsed_ingredients.append((ingredient.strip(), ''))
    return parsed_ingredients

In [32]:
# 連接到 Neo4j Aura 並創建圖譜
NEO4J_URI = "neo4j+s://f0372b25.databases.neo4j.io"
NEO4J_USER = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

# 添加食譜和成分到圖譜中
for index, row in test_recipe.iterrows():
    recipe_title = row['Title']
    recipe_instructions = row['Instructions']
    recipe_node = {"title": recipe_title, "instructions": recipe_instructions}  # 將屬性存儲在字典中
    G.add_node(recipe_title, **recipe_node)  # 使用屬性字典添加節點

    ingredients = parse_ingredients(row['Cleaned_Ingredients'])
    for name, amount in ingredients:
        ingredient_node = {"name": name.lower()}
        if not G.has_node(name.lower()):
            G.add_node(name.lower(), **ingredient_node)  # 使用屬性字典添加節點
        G.add_edge(recipe_title, name.lower(), relationship="CONTAINS", amount=amount)

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

def clear_database(tx):
    tx.run("MATCH (n) DETACH DELETE n")

def create_node(tx, label, properties):
    tx.run(f"CREATE (n:{label} $properties)", properties=properties)

def create_relationship(tx, start_node, end_node, relationship, properties):
    tx.run(f"""
    MATCH (a:{start_node[0]} {{title: $start_title}})
    MATCH (b:{end_node[0]} {{name: $end_name}})
    CREATE (a)-[r:{relationship} $properties]->(b)
    """, start_title=start_node[1]['title'], end_name=end_node[1]['name'], properties=properties)

# 清空現有數據（選擇性）
with driver.session() as session:
    session.write_transaction(clear_database)

# 添加節點和邊到 Neo4j
with driver.session() as session:
    for node_key in G.nodes:
        node = G.nodes[node_key]  # 直接檢索節點屬性字典
        if 'instructions' in node:
            session.write_transaction(create_node, "Recipe", {"title": node_key, "instructions": node['instructions']})
        else:
            session.write_transaction(create_node, "Ingredient", {"name": node_key})

    for edge in G.edges(data=True):
        recipe_node = ("Recipe", {"title": edge[0]})
        ingredient_node = ("Ingredient", {"name": edge[1]})
        session.write_transaction(create_relationship, recipe_node, ingredient_node, edge[2]['relationship'], {"amount": edge[2]['amount']})

print("知識圖譜建立完成！")

/var/folders/zc/fpd0g53d2vgdp8kf_k0xv7h40000gn/T/ipykernel_19637/626722458.py:37: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(clear_database)
/var/folders/zc/fpd0g53d2vgdp8kf_k0xv7h40000gn/T/ipykernel_19637/626722458.py:44: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_node, "Recipe", {"title": node_key, "instructions": node['instructions']})
/var/folders/zc/fpd0g53d2vgdp8kf_k0xv7h40000gn/T/ipykernel_19637/626722458.py:46: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_node, "Ingredient", {"name": node_key})
/var/folders/zc/fpd0g53d2vgdp8kf_k0xv7h40000gn/T/ipykernel_19637/626722458.py:51: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_relationship, recipe_node, ingredient_node, edge[2]['relationship'], {"amount": edge[2]['amount']})


知識圖譜建立完成！


In [38]:
vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated procedure. ('db.create.setVectorProperty' has been replaced by 'db.create.setNodeVectorProperty')} {position: line: 1, column: 70, offset: 69} for query: "UNWIND $data AS row MATCH (n:`Document`) WHERE elementId(n) = row.id CALL db.create.setVectorProperty(n, 'embedding', row.embedding) YIELD node RETURN count(*)"


In [45]:
graph = Neo4jGraph()
# Retriever

graph.query(
    "CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

# Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All the recipe, food, ingredient, nutrition that "
        "appear in the text",
    )

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)
# llm=ChatOllama(model="llama3")
llm=ChatOpenAI(temperature=0, model_name="gpt-4o") 
entity_chain = prompt | llm.with_structured_output(Entities)

In [47]:
entity_chain.invoke({"question": "烤馬鈴薯怎麼做？"}).names

['烤馬鈴薯']

In [58]:
def generate_full_text_query(input: str) -> str:
    """
    Generate a full-text search query for a given input string.

    This function constructs a query string suitable for a full-text search.
    It processes the input string by splitting it into words and appending a
    similarity threshold (~2 changed characters) to each word, then combines
    them using the AND operator. Useful for mapping entities from user questions
    to database values, and allows for some misspelings.
    """
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()

# Fulltext index query
def structured_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            CALL {
                WITH node
                MATCH (i:Ingredient)<-[:CONTAINS]-(r:Recipe)
                WHERE i.name IN $ingredient_list
                RETURN r.title + ' - ' + r.instructions AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [49]:
def retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ". join(unstructured_data)}
    """
    return final_data

In [50]:
# Condense a chat history and follow-up question into a standalone question
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question,
in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""  # noqa: E501
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

_search_query = RunnableBranch(
    # If input includes chat_history, we condense it with the follow-up question
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),  # Condense follow-up question and chat into a standalone_question
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | ChatOpenAI(temperature=0)
        | StrOutputParser(),
    ),
    # Else, we have no chat history, so just pass through the question
    RunnableLambda(lambda x : x["question"]),
)

In [51]:
template = """請回答下方問題
{context}

問題: {question}
請使用中文回答
答案:"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

In [52]:
chain.invoke({"question": "how to cook Potatoes?"})

Search query: how to cook Potatoes?


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: MENTIONS)} {position: line: 5, column: 32, offset: 166} for query: "CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})\n            YIELD node,score\n            CALL {\n              WITH node\n              MATCH (node)-[r:!MENTIONS]->(neighbor)\n              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output\n              UNION ALL\n              WITH node\n              MATCH (node)<-[r:!MENTIONS]-(neighbor)\n              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output\n            

'要煮土豆，可以按照以下步驟進行：\n\n1. **選擇土豆**：選擇新鮮、無損壞的土豆。\n\n2. **清洗土豆**：用清水徹底清洗土豆，去除表面的泥土和雜質。\n\n3. **去皮（可選）**：根據個人口味決定是否去皮。如果不去皮，請確保清洗乾淨。\n\n4. **切塊**：將土豆切成均勻的小塊，這樣可以確保煮的時候受熱均勻。\n\n5. **煮土豆**：\n   - 將切好的土豆放入鍋中，加入足夠的水，水量應該能完全覆蓋土豆。\n   - 加入一小撮鹽，這樣可以增加土豆的風味。\n   - 開大火煮沸，然後轉中小火繼續煮約10-15分鐘，直到土豆變軟。可以用叉子或筷子插入土豆，如果能輕鬆插入，說明土豆已經煮熟。\n\n6. **瀝乾水分**：將煮熟的土豆用濾網瀝乾水分。\n\n7. **進一步處理（可選）**：煮熟的土豆可以直接食用，也可以進行進一步的烹飪，比如炒、烤、做成土豆泥等。\n\n希望這些步驟能幫助你煮出美味的土豆！'